In [37]:
pip install cpmpy

You should consider upgrading via the '/home/fosco/PycharmProjects/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [70]:
import json
import cpmpy as cp
import itertools

In [82]:
ub = int(1e4)
M = int(1e5)
with open("data/model.json") as json_file:
    items = json.load(json_file)

jobs = items['nodes']
constraint = items['edges']
dict_seq = {}
info = {}
model = cp.Model()

In [83]:
size=0
#Mapping id to index od decision variable
for job,job_info in jobs.items():
    id = job_info['__id__']
    resource = job_info['properties']['Resource']['value']
    info[id] = [size,resource]
    size+=1

start = cp.intvar(0,ub,shape=size, name="start")
duration = cp.intvar(0,ub,shape=size, name="dur")
end = cp.intvar(0,ub, shape=size, name="end")
precedences = cp.boolvar(shape=(size,size))
makespan =  cp.intvar(0,ub)

In [84]:
#Introducing time constraints
# for (comb_1,comb_2) in (itertools.combinations(range(size), 2)):
#     model += (precedences[comb_1,comb_2] + precedences[comb_2,comb_1] == 1)
#     model += ((start[comb_2] + duration[comb_2] - M * precedences[comb_1,comb_2]) <= start[comb_1])



#Fixed duration
for job,job_info in jobs.items():
    val = job_info['properties'][f'{job}_duration']['value']
    if val != None:
        model += (duration[info[job_info['__id__']][0]] == val)


for constr,constr_info in constraint.items():
    #ILP FORMULATION
    # if constr_info['__type__'] == 'HasNoOverlap':
    #     first_subject = info[constr_info['first']['__id__']][0]
    #     second_subject = info[constr_info['second']['__id__']][0]
    #     model += ((start[first_subject] + duration[first_subject] - M*precedences[second_subject,first_subject]) <= start[second_subject])
    #CP FORMULATION
    if constr_info['__type__'] == 'HasNoOverlap':
        first_subject = info[constr_info['first']['__id__']][0]
        second_subject = info[constr_info['second']['__id__']][0]
        model += cp.NoOverlap([start[first_subject], start[second_subject]],
                              [duration[first_subject], duration[second_subject]],
                              [end[first_subject], end[second_subject]])
    if constr_info['__type__'] == 'IsRightAfter':
        first_subject = info[constr_info['first']['__id__']][0]
        second_subject = info[constr_info['second']['__id__']][0]
        model += ((start[second_subject] + duration[second_subject]) == start[first_subject])
    if constr_info['__type__'] == 'IsSynced':
        first_subject = info[constr_info['first']['__id__']][0]
        second_subject = info[constr_info['second']['__id__']][0]
        model += (start[first_subject] == start[second_subject])
        model += (duration[first_subject] == duration[second_subject])
    if constr_info['__type__'] == 'IsRightBefore':
        first_subject = info[constr_info['first']['__id__']][0]
        second_subject = info[constr_info['second']['__id__']][0]
        model += ((start[first_subject] + duration[first_subject]) == start[second_subject])


In [85]:
#definition makespan as maximum start + duration
# for i in range(size):
#     model += (start[i] + duration[i] <= makespan)
#minimization of objective value
model.minimize(cp.max(start + duration))

In [90]:
print("is there a solution?", model.solve())

is there a solution? True


In [89]:
print(cp.max(start + duration).value())

2764


In [77]:
from cpmpy.tools.mus import mus

for c in mus(model.constraints):
    print("-",c)

- no_overlap([start[67], start[88]],[dur[67], dur[88]],[end[67], end[88]])
- ((start[67]) + (dur[67])) == (start[68])
- ((start[77]) + (dur[77])) == (start[80])
- ((start[80]) + (dur[80])) == (start[79])
- ((start[79]) + (dur[79])) == (start[82])
- ((start[82]) + (dur[82])) == (start[81])
- ((start[81]) + (dur[81])) == (start[83])
- ((start[88]) + (dur[88])) == (start[89])
- ((start[98]) + (dur[98])) == (start[101])
- ((start[101]) + (dur[101])) == (start[100])
- ((start[100]) + (dur[100])) == (start[103])
- ((start[103]) + (dur[103])) == (start[102])
- ((start[102]) + (dur[102])) == (start[104])
- (start[77]) == (start[68])
- (dur[77]) == (dur[68])
- (dur[79]) == (dur[69])
- (dur[81]) == (dur[71])
- (start[98]) == (start[89])
- (dur[98]) == (dur[89])
- (dur[100]) == (dur[90])
- (dur[102]) == (dur[92])
- dur[67] == 320
- dur[68] == 192
- dur[69] == 152
- dur[71] == 96
- dur[88] == 320
- dur[89] == 192
- dur[90] == 152
- dur[92] == 96
